In [4]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [5]:
train_all_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_all_df.reset_index(inplace=True)
train_all_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,5,5263,555,1,1,0,1
1,2,2015-07-31,5,6064,625,1,1,0,1
2,3,2015-07-31,5,8314,821,1,1,0,1
3,4,2015-07-31,5,13995,1498,1,1,0,1
4,5,2015-07-31,5,4822,559,1,1,0,1


In [6]:
def featurize(df):
    df = df.copy()
    def featurizeStateHoliday(x):
        if x == 'a':
            return 1
        elif x == 'b':
            return 2
        elif x == 'c':
            return 3
        else:
            return x
    df['StateHoliday'] = df['StateHoliday'].map(lambda x: featurizeStateHoliday(x))
    

    def featurizeDate(ts): 
        features = {}
        features['cal:dayOfMonth'] = ts.day
        features['cal:dayOfWeek'] = ts.dayofweek
        features['cal:dayofyear'] = ts.dayofyear
        features['cal:month'] = ts.month
        features['cal:year'] = ts.year
        features['cal:weekofyear'] =  ts.weekofyear
        features['cal:quarter'] = ts.quarter

        return features

    date_features_series = df['Date'].map(lambda x: featurizeDate(x))
    date_features = DataFrame.from_records(date_features_series.tolist(), index=df['Date'])

    df = pandas.concat([df, date_features.reset_index()], axis=1)

    del df['Date']
    
    return df

In [7]:
train_all_df = featurize(train_all_df)

train_df, test_df = cross_validation.train_test_split(train_all_df, train_size=.9)

# train_individual_df, train_ensemble_df = cross_validation.train_test_split(train_df, train_size=.5)

train_all_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,1,5,5263,555,1,1,0,1,31,4,212,7,3,31,2015
1,2,5,6064,625,1,1,0,1,31,4,212,7,3,31,2015
2,3,5,8314,821,1,1,0,1,31,4,212,7,3,31,2015
3,4,5,13995,1498,1,1,0,1,31,4,212,7,3,31,2015
4,5,5,4822,559,1,1,0,1,31,4,212,7,3,31,2015


In [8]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

# One Big Model

In [9]:
features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
regressor_big = ExtraTreesRegressor(n_estimators=100, n_jobs=-1)
regressor_big.fit(train_df[features], train_df['Sales'])
score(predicted=regressor_big.predict(test_df[features]), actual=test_df['Sales'])

0.5560559625926835

# Per store model


In [10]:

regressors = {}
features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
i = 0
for store in train_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_train_df = train_df[train_df['Store'] == store]
    
    regressor = ExtraTreesRegressor(n_estimators=50, n_jobs=-1)
    regressor.fit(store_train_df[features], store_train_df['Sales'])
    regressors[store] = regressor


0
100
200
300
400
500
600
700
800
900
1000
1100


In [11]:
store_predictions = {}
for store in test_df['Store'].unique():
    
    features = [col for col in test_df.columns if col != 'Sales' and col != 'Customers']
    store_test_df = test_df[test_df['Store'] == store]
    store_predictions[store] = DataFrame.from_dict({"Predicted": regressors[store].predict(store_test_df[features]), "Actual": store_test_df['Sales']})

In [12]:
results = pandas.concat(store_predictions.values())

In [13]:
score(predicted=results.Predicted, actual=results.Actual)

0.14669575041920244

# Means

In [14]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


train_df = pandas.merge(train_df, calculate_means(train_df), on = ['Store','DayOfWeek','Promo'], how='left')

train_df['Predict-mean'] = train_df['Sales_y']
del train_df['Sales_y']
train_df['Sales'] = train_df['Sales_x']
del train_df['Sales_x']

train_df.fillna(train_df['Predict-mean'].mean(), inplace=True)



In [15]:
train_df.head()

,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year,Predict-mean,Sales
0,233,2,1348,1,1,0,0,17,1,351,12,4,51,2013,11262.875000,14887
1,314,7,0,0,0,0,0,29,6,88,3,1,13,2015,0.000000,0
2,524,7,201,1,0,0,0,12,6,285,10,4,41,2014,1324.032787,1791
3,978,1,1013,1,1,0,0,3,0,154,6,2,23,2013,10660.714286,7287
4,60,2,684,1,0,0,0,12,1,316,11,4,46,2013,6689.783333,6904


In [1]:
^^^^ STOP: inspect train_df, make sure all three sales predictions are in place.
    NEXT: continue on with ensemble and test
    NEXT: add in customer predictions

SyntaxError: invalid syntax (<ipython-input-1-a975497927ee>, line 1)

# Ensemble



In [2]:
train_ensemble_df_bk = train_ensemble_df.copy()

NameError: name 'train_ensemble_df' is not defined

In [ ]:
# Create ensemble training set
partials = []

features = [col for col in train_ensemble_df.columns if col != 'Sales' and col != 'Customers']
for store in train_ensemble_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_train_df = train_individual_df[train_individual_df['Store'] == store]
    store_train_df['Predict-all_in_one'] = regressor_big.predict(store_train_df[features])
    store_train_df['Predict-by_store'] = regressors[store].predict(store_train_df[features])
    
    partials.append(store_train_df)
    

In [ ]:
ensemble_df = pandas.concat(partials)

In [ ]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


ensemble_df = pandas.merge(ensemble_df, calculate_means(ensemble_df), on = ['Store','DayOfWeek','Promo'], how='left')

ensemble_df['Predict-mean'] = ensemble_df['Sales_y']
del ensemble_df['Sales_y']
ensemble_df['Sales'] = ensemble_df['Sales_x']
del ensemble_df['Sales_x']

ensemble_df.fillna(ensemble_df['Predict-mean'].mean(), inplace=True)


In [ ]:
partials = []

features = [col for col in test_df.columns if col != 'Sales' and col != 'Customers']
for store in test_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_test_df = test_df[test_df['Store'] == store]
    store_test_df['Predict-all_in_one'] = regressor_big.predict(store_test_df[features])
    store_test_df['Predict-by_store'] = regressors[store].predict(store_test_df[features])
    
    partials.append(store_test_df)
test_ensemble_df = pandas.concat(partials)

In [ ]:
features = [col for col in ensemble_df.columns if col != 'Sales' and col != 'Customers']
regressor_ensemble = LinearRegression(normalize="L1")
regressor_ensemble.fit(ensemble_df[features], ensemble_df['Sales'])

In [ ]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


test_ensemble_df = pandas.merge(test_ensemble_df, calculate_means(ensemble_df), on = ['Store','DayOfWeek','Promo'], how='left')

test_ensemble_df['Predict-mean'] = test_ensemble_df['Sales_y']
del test_ensemble_df['Sales_y']
test_ensemble_df['Sales'] = test_ensemble_df['Sales_x']
del test_ensemble_df['Sales_x']

test_ensemble_df.fillna(test_ensemble_df['Predict-mean'].mean(), inplace=True)


In [ ]:
# Fix order
test_ensemble_df = test_ensemble_df[ensemble_df.columns]

In [ ]:
features = [col for col in test_ensemble_df.columns if col != 'Sales' and col != 'Customers']
score(predicted=regressor_ensemble.predict(test_ensemble_df[features]), actual=test_ensemble_df['Sales'])

# Submit

In [ ]:
eval_df = DataFrame.from_csv("test.csv", parse_dates=['Date'])

In [ ]:
eval_df.reset_index(inplace=True)
eval_df.head()

In [3]:
eval_df = featurize(eval_df)


NameError: name 'featurize' is not defined

In [ ]:
eval_df.columns

In [ ]:
eval_df.fillna(0, inplace=True)

In [ ]:
partials = []

features = [col for col in eval_df.columns if col != 'Sales' and col != 'Customers' and col != "Id"]
for store in eval_df['Store'].unique():

    if (i % 100 == 0):
        print (i)
    i = i + 1
    
    store_test_df = eval_df[eval_df['Store'] == store]
    store_test_df['Predict-all_in_one'] = regressor_big.predict(store_test_df[features])
    store_test_df['Predict-by_store'] = regressors[store].predict(store_test_df[features])
    
    partials.append(store_test_df)
eval_df = pandas.concat(partials)

In [ ]:
del eval_df['Sales_x']
del eval_df['Sales_y']

In [ ]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means


eval_df = pandas.merge(eval_df, calculate_means(train_all_df), on = ['Store','DayOfWeek','Promo'], how='left')

eval_df['Predict-mean'] = eval_df['Sales']
del eval_df['Sales']

eval_df.fillna(eval_df['Predict-mean'].mean(), inplace=True)


In [ ]:
features = [col for col in test_ensemble_df.columns if col != 'Sales' and col != 'Customers']
predictions = regressor_ensemble.predict(eval_df[features])

In [ ]:
eval_df['Predictions'] = predictions

In [ ]:
ensemble_results = eval_df[['Id', 'Predictions']]
ensemble_results.columns=['Id', 'Sales']
ensemble_results.to_csv( "results-ensemble.csv", index = False )